In [1]:
# pip install py7zr
# pip install transformers
# pip install torch
# pip install beautifulsoup4


In [2]:
import os
os.chdir("data")

In [3]:
def download_subject(subject):
    url = "https://archive.org/download/stackexchange/{}.stackexchange.com.7z".format(subject)
    filename = "{}.stackexchange.com.7z".format(subject)

    response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(response.content)

    return filename

def unzip(zip_filename):
    # unzip .7z file
    import py7zr

    # Specify the name of the input 7z file and the output folder
    input_file = zip_filename
    output_folder = zip_filename[:-4]

    # Make sure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Open the 7z file and extract all its contents to the output folder
    with py7zr.SevenZipFile(input_file, 'r') as archive:
        archive.extractall(output_folder)
    
    return output_folder

def extract_dataset(subject_folder, subject, csv_filename):
    # read posts.xml
    import xml.etree.ElementTree as ET
    import csv

    # Parse the XML file
    tree = ET.parse(subject_folder + '/Posts.xml')
    root = tree.getroot()

    # Open the output CSV file
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # Write the header row
        writer.writerow(['id', 'title', 'body', 'tags'])
        
        # Loop over all the rows in the XML file
        for row in root.findall('row'):
            # Extract the relevant fields from the XML element
            post_id = row.get('Id')
            title = row.get('Title')
            body = row.get('Body')
            tags = row.get('Tags')
            
            # Write the row to the CSV file
            writer.writerow([post_id, title, body, tags])

    return csv_filename


In [19]:
import requests

subjects=["datascience", "money"] #, "music", "politics", "raspberrypi", "scifi", "skeptics", "unix"]

for subject in subjects:
    zip_filename = download_subject(subject)
    subject_folder = unzip(zip_filename)
    csv_filename = subject + '_posts.csv'
    extract_dataset(subject_folder, subject, csv_filename)

In [17]:
# df['tags'].str.split('><').apply(lambda x: [tag.strip("<> ") for tag in x if tag.strip()] if isinstance(x, list) else [])
import pandas as pd, numpy as np
from bs4 import BeautifulSoup
import re

# Define a function to extract text content from HTML tags
def pre_process(text):
    if not isinstance(text, str) or text.strip() == '':
        return ''
    text = BeautifulSoup(text).get_text()
    # fetch alphabetic characters
    text = re.sub("[^a-zA-Z]", " ", text)
    # convert text to lower case
    text = text.lower()
    # split text into tokens to remove whitespaces
    tokens = text.split()
    return " ".join(tokens)
    #   soup = BeautifulSoup(html, 'html.parser')
    # return soup.get_text()

Exploring only the datascience_posts here:

In [39]:
import pandas as pd

# csv name
csv_filename = "datascience" + '_posts.csv'

# Load CSV file into Pandas DataFrame
df = pd.read_csv(csv_filename)

# Create content and labels lists
content = df['body'].apply(pre_process).tolist()
labels = df['tags'].str.split('><').apply(lambda x: [tag.strip("<> ") for tag in x if tag.strip()] if isinstance(x, list) else [])

# Print the first content and labels for verification
print(content[0])
print(labels[0])

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
train_content, test_content, train_labels, test_labels = train_test_split(content, labels, test_size=0.2, random_state=42)

# Print the number of samples in each set
print('Number of training samples:', len(train_content))
print('Number of testing samples:', len(test_content))

/tmp/ipykernel_91751/1696799192.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text).get_text()


i ve always been interested in machine learning but i can t figure out one thing about starting out with a simple hello world example how can i avoid hard coding behavior for example if i wanted to teach a bot how to avoid randomly placed obstacles i couldn t just use relative motion because the obstacles move around but i don t want to hard code say distance because that ruins the whole point of machine learning obviously randomly generating code would be impractical so how could i do this
['machine-learning']
Number of training samples: 59461
Number of testing samples: 14866


In [1]:
# Bert direction...
# import torch
# from sklearn.preprocessing import MultiLabelBinarizer
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


# # Encode the tags as one-hot vectors
# mlb = MultiLabelBinarizer()
# y_train = mlb.fit_transform(train_labels)

# # Tokenize the posts using the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# def tokenize_function(c):
#     return tokenizer(c, padding="max_length", truncation=True)

# X_train = tokenizer.batch_encode_plus(
#         train_content,
#         return_attention_mask=True,
#         return_token_type_ids=True,
#         pad_to_max_length=True,
#         padding='max_length',
#         truncation=True

#     )
    
    
# # Define the BERT model for sequence classification
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(mlb.classes_))

# # Define the training arguments and trainer
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     evaluation_strategy='steps',
#     eval_steps=50,
#     load_best_model_at_end=True
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=torch.utils.data.TensorDataset(torch.tensor(X_train['input_ids']), torch.tensor(X_train['attention_mask']), torch.tensor(y_train))
# )

# # Train the model
# trainer.train()

# # Save the model and tokenizer
# model.save_pretrained('./model')
# tokenizer.save_pretrained('./model')